In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import warnings

In [2]:
warnings.filterwarnings("ignore")
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
gen_df = pd.read_csv('./data/Plant_1_Generation_Data.csv')
weather_df = pd.read_csv('./data/Plant_1_Weather_Sensor_Data.csv')

gen_df['DATE_TIME'] = pd.to_datetime(gen_df['DATE_TIME'], format='%d-%m-%Y %H:%M')
weather_df['DATE_TIME'] = pd.to_datetime(weather_df['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')

gen_agg = gen_df.groupby('DATE_TIME').agg({
    'DC_POWER': 'sum',
    'AC_POWER': 'sum',
    'DAILY_YIELD': 'mean',
    'TOTAL_YIELD': 'mean'
}).reset_index()

df = pd.merge(gen_agg, weather_df, on='DATE_TIME')
df = df.drop(columns=['PLANT_ID', 'SOURCE_KEY'])

df['HOUR'] = df['DATE_TIME'].dt.hour
df['DAY_SIN'] = np.sin(2*np.pi*df['HOUR']/24)
df['DAY_COS'] = np.cos(2*np.pi*df['HOUR']/24)

In [ ]:
lookback = 6  
forecast_horizon = 6  
test_size = 0.2

features = ['IRRADIATION', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'DAY_SIN', 'DAY_COS']
target = 'AC_POWER'

X, y = [], []
for i in range(len(df) - lookback - forecast_horizon + 1):
    X.append(df[features].values[i:i+lookback])
    y.append(df[target].values[i+lookback:i+lookback+forecast_horizon])
X, y = np.array(X), np.array(y)

split_idx = int(len(X) * (1 - test_size))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

x_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = x_scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1]))
X_train_scaled = X_train_scaled.reshape(-1, lookback, len(features))

X_test_scaled = x_scaler.transform(X_test.reshape(-1, X_test.shape[-1]))
X_test_scaled = X_test_scaled.reshape(-1, lookback, len(features))

y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

train_dataset = TensorDataset(torch.FloatTensor(X_train_scaled), torch.FloatTensor(y_train_scaled))
test_dataset = TensorDataset(torch.FloatTensor(X_test_scaled), torch.FloatTensor(y_test_scaled))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
len(features)

5

In [ ]:
class SolarLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=6):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 32),
            nn.ReLU(),
            nn.Linear(32, output_size)
        )
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SolarLSTM(input_size=len(features)).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
epochs = 100

for epoch in range(epochs):
    model.train()
    train_loss = 0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            val_loss += criterion(outputs, batch_y).item()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss/len(train_loader):.4f} | Val Loss: {val_loss/len(test_loader):.4f}")

import os 
import joblib

os.makedirs("Solar_LSTM", exist_ok=True)
joblib.dump(x_scaler, './Solar_LSTM/x_scaler.joblib') 
joblib.dump(y_scaler, './Solar_LSTM/y_scaler.joblib')
torch.save(model.state_dict(), "./Solar_LSTM/solar_lstm_model.pth")

Epoch 10/100 | Train Loss: 0.1005 | Val Loss: 0.0902
Epoch 20/100 | Train Loss: 0.0924 | Val Loss: 0.0858
Epoch 30/100 | Train Loss: 0.0888 | Val Loss: 0.0905
Epoch 40/100 | Train Loss: 0.0882 | Val Loss: 0.0978
Epoch 50/100 | Train Loss: 0.0854 | Val Loss: 0.0894
Epoch 60/100 | Train Loss: 0.0819 | Val Loss: 0.1023
Epoch 70/100 | Train Loss: 0.0803 | Val Loss: 0.0938
Epoch 80/100 | Train Loss: 0.0761 | Val Loss: 0.0997
Epoch 90/100 | Train Loss: 0.0750 | Val Loss: 0.1005
Epoch 100/100 | Train Loss: 0.0708 | Val Loss: 0.1135
